In [ ]:
from pathlib import Path
import os
import sys

current_path = Path.cwd()
if current_path.name == "notebooks":
    project_root = current_path.parent
else:
    project_root = current_path

os.chdir(project_root)
print(f"Current working directory: {Path.cwd()}")

sys.path.append(str(project_root / "src"))


# 03 - Feature Engineering

## Objetivo
Construcción y transformación de variables predictivas.

## 1. Transformación de tipos y columnas derivadas

## 2. Codificación de variables categóricas

## 3. Normalización y escalado

## 4. Dataset final para modelado

## 5. Guardado del dataset limpio